In [0]:
!pip install --no-cache-dir -I pillow
!pip install hdf5storage
!pip install http://download.pytorch.org/whl/cu92/torch-1.1.0-cp36-cp36m-linux_x86_64.whl
!pip3 install torchvision
#!git clone https://github.com/lanpa/tensorboardX && cd tensorboardX && python setup.py install
!pip install tensorboardX


In [0]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir("/content/drive/My Drive/app/folder_name/") # folder containing your images used for training and testing 
path=os.getcwd() 

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
### 
from __future__ import division
from scipy import interpolate
import random
import os
import os.path
import h5py
import cv2
import glob
import numpy as np

import PIL

import scipy.io
import matplotlib
matplotlib.use('Agg')
from matplotlib.backends.backend_agg import FigureCanvasAgg
from matplotlib.figure import Figure

import torchvision.utils as utils
from tensorboardX import SummaryWriter
###
import torch
import torch.nn as nn
import argparse
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torch.utils.data import DataLoader
from torch.autograd import Variable
import torch.utils.data as udata
from torch.utils.data.sampler import SubsetRandomSampler
import torchvision.utils as utils
import time
import scipy.io as sio
import logging
import hdf5storage
import datetime
from math import sqrt


In [0]:

class AverageMeter(object):
    """Computes and stores the average and current value."""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def save_checkpoint_sp(model_path, epoch, iteration, model, optimizer, evaluation):
    """Save the checkpoint."""
    state = {
            'epoch': epoch,
            'iter': iteration,
            'state_dict': model.state_dict(),
            'optimizer' : optimizer.state_dict(),
            }
    
    torch.save(state, os.path.join(model_path, 'hscnn_6layer_dim10_{}.pkl'.format(evaluation)))

def plot_spectrum(real, fake, epoch, i):
    x =np.linspace(397.32, 1003.58, 204, endpoint=True) # the wavebands of the hyperspectral image
    fig = Figure()
    canvas = FigureCanvasAgg(fig)
    ax = fig.gca()
    plot_real,  = ax.plot(x, real)
    plot_fake,  = ax.plot(x, fake)
    fig.legend((plot_real,plot_fake), ('real', 'fake'))
    canvas.draw()
    fig.savefig(os.path.join(iteration_path, "{}_test_{}.png".format(epoch,i)))
    I = np.fromstring(canvas.tostring_rgb(), dtype='uint8', sep='')
    I = I.reshape(canvas.get_width_height()[::-1]+(3,))
    I = np.transpose(I, [2,0,1])
    return np.float32(I)

def initialize_logger(file_dir):
    """Print the results in the log file."""
    logger = logging.getLogger()
    fhandler = logging.FileHandler(filename=file_dir, mode='a')
    formatter = logging.Formatter('%(asctime)s - %(message)s',"%Y-%m-%d %H:%M:%S")
    fhandler.setFormatter(formatter)
    logger.addHandler(fhandler)
    logger.setLevel(logging.INFO)
    return logger

def save_matv73(mat_name, var_name, var):
    hdf5storage.savemat(mat_name, {var_name: var}, format='7.3', store_python_metadata=True)

    
def get_reconstruction(input, num_split, dimension, model):
    """As the limited GPU memory split the input."""
    input_split = torch.split(input,  int(input.shape[3]/num_split), dim=dimension)
    output_split = []
    for i in range(num_split):
        var_input = Variable(input_split[i].cuda())
        var_output = model(var_input)
        output_split.append(var_output.data)
        if i == 0:
            output = output_split[i]
        else:
            output = torch.cat((output, output_split[i]), dim=dimension)
    
    return output


def reconstruction(rgb,model):
    """Output the final reconstructed hyperspectral images."""
    img_res = get_reconstruction(torch.from_numpy(rgb).float(),1, 3, model)
    img_res = img_res.cpu().numpy()*1
    img_res = np.transpose(np.squeeze(img_res))
    img_res_limits = np.minimum(img_res,1)
    img_res_limits = np.maximum(img_res_limits,0)
    return img_res_limits

def mrae_loss(im_true, im_fake):
    error = torch.abs(im_fake-im_true)/im_true
    rrmse = torch.mean(error.view(-1))
    return rrmse


In [0]:
## new fucntion for process data
path=os.getcwd() 

def normalize(data):
    return data/np.amax(data)


def data_process(path=path):
    NO. = 1
    h5f = h5py.File('train_tomato.h5', 'w')  ## the file concatenated hsi and rgb in one file
    filenames_hyper = glob.glob(os.path.join(path,'tomato_hsi','*.h5'))
    filenames_rgb = glob.glob(os.path.join(path,'tomato_hsi','*.png'))
    filenames_hyper.sort()
    filenames_rgb.sort()
    for i in range(len(filenames_hyper)):
        print("\n")
        print(filenames_hyper[i], filenames_rgb[i])
        # load hyperspectral image
        mat =  h5py.File(filenames_hyper[i],'r')
        hyper = np.float32(np.array(mat['img']))
#            hyper.shape
        hyper = np.transpose(hyper, [0,2,1])
        hyper = normalize(hyper)
        print("hyper_test_max", np.amax(hyper))
        print("hyper_test_min",np.amin(hyper))
        mat.close()
        # load rgb image
        rgb =  cv2.imread(filenames_rgb[i])
        rgb=cv2.cvtColor(rgb, cv2.COLOR_BGR2RGB)
        
        rgb = np.transpose(rgb, [2,0,1])

        rgb = normalize(np.float32(rgb))
        print("rgb_test_max", np.amax(np.float32(rgb)))
        print("rgb_test_min", np.amin(np.float32(rgb)))
#            mat.close()
            # creat patches
        data = np.concatenate((hyper,rgb), 0)
        h5f.create_dataset(str(NO.), data=data)
        NO. += 1
    h5f.close()
    print("NO. of samples: {}".format(NO.-1))

class HyperDataset(udata.Dataset):
    def __init__(self, crop_size=64):
        self.crop_size = crop_size
        h5f = h5py.File('train_tomato.h5', 'r')
        self.keys = list(h5f.keys())
        keys.sort()
        h5f.close()
    def __len__(self):
        return len(self.keys)
    def __getitem__(self, index):
        h5f = h5py.File('train_tomato.h5', 'r')

        key = str(self.keys[index])
        data = np.array(h5f[key])
        data = torch.Tensor(data)
        # crop
        w = int(data.size()[1])
        h = int(data.size()[2])
        th, tw = self.crop_size, self.crop_size
        if w > tw or h > th:
                i = 0
                j = 0
            data = data[:,i:i+th,j:j+tw]
        h5f.close()
        return data[0:204,:,:], data[204:207,:,:]

def batch_MRAE(im_true, im_fake, data_range=255.):
    N = im_true.size()[0]
    C = im_true.size()[1]
    H = im_true.size()[2]
    W = im_true.size()[3]
    Itrue = im_true.clamp(0.,1.).mul_(data_range).reshape(N, C*H*W)
    Ifake = im_fake.clamp(0.,1.).mul_(data_range).reshape(N, C*H*W)
    mse = nn.MSELoss(reduce=False)
    err = mse(Itrue, Ifake).sqrt_().div_(Itrue).sum(dim=1, keepdim=True).div_(C*H*W)
    return torch.mean(err)

def batch_RMSE(im_true, im_fake, data_range=255.):
    N = im_true.size()[0]
    C = im_true.size()[1]
    H = im_true.size()[2]
    W = im_true.size()[3]
    Itrue = im_true.clamp(0.,1.).mul_(data_range).reshape(N, C*H*W)
    Ifake = im_fake.clamp(0.,1.).mul_(data_range).reshape(N, C*H*W)
    mse = nn.MSELoss(reduce=False)
    err = mse(Itrue, Ifake).sum(dim=1, keepdim=True).div_(C*H*W).sqrt_()
    return torch.mean(err)

def batch_SAM(im_true, im_fake):
    N = im_true.size()[0]
    C = im_true.size()[1]
    H = im_true.size()[2]
    W = im_true.size()[3]
    Itrue = im_true.clone().reshape(N, C, H*W)
    Ifake = im_fake.clone().reshape(N, C, H*W)
    nom = torch.mul(Itrue, Ifake).sum(dim=1).reshape(N, H*W)
    denom1 = torch.pow(Itrue,2).sum(dim=1).sqrt_().reshape(N, H*W)
    denom2 = torch.pow(Ifake,2).sum(dim=1).sqrt_().reshape(N, H*W)
    sam = torch.div(nom, torch.mul(denom1, denom2)).acos_().reshape(N, H*W)
    sam = sam / np.pi * 180
    sam = torch.sum(sam) / (N*H*W)
    return sam


def weights_init_kaimingNormal(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.kaiming_normal(m.weight.data, a=0.2, mode='fan_in')
    elif classname.find('Linear') != -1:
        nn.init.kaiming_normal(m.weight.data, a=0.2, mode='fan_in')
    elif classname.find('BatchNorm') != -1:
        nn.init.normal(m.weight.data, 0, 0.01)
        nn.init.constant(m.bias.data, 0.0)
    elif classname.find('InstanceNorm') != -1:
        nn.init.normal(m.weight.data, 0, 0.01)
        nn.init.constant(m.bias.data, 0.0)



In [0]:
#####             resblock

def conv3x3(in_channels, out_channels):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3, 
                     stride=1, padding=1, bias=True)
    
class resblock(nn.Module):
    def __init__(self, block, block_num, input_channel, output_channel):
        super(resblock, self).__init__()

        self.in_channels = input_channel
        self.out_channels = output_channel
        self.input_conv = conv3x3(self.in_channels, out_channels=64)  
        self.conv_seq = self.make_layer(block, block_num)
        self.conv = conv3x3(64, 64)
        self.relu = nn.ReLU(inplace=True)
        self.output_conv = conv3x3(in_channels=64,  out_channels=self.out_channels)
        
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n=m.kernel_size[0]*m.kernel_size[1]*m.out_channels
                m.weight.data.normal_(0,sqrt(2./n))# the devide  2./n  carefully  
                
    def make_layer(self,block,num_layers):
        layers = []
        for i in range(num_layers):
            layers.append(block()) # there is a () 
        return nn.Sequential(*layers)   
    
    def forward(self, x):
       
        out = self.input_conv(x)
        residual = out
        out = self.conv_seq(out)
        out = self.conv(out)
        out = torch.add(out,residual)  
        out = self.relu(out)
        out = self.output_conv(out)
        return out

# Learning rate
def poly_lr_scheduler(optimizer, init_lr, epoch, lr_decay_iter=1,
                      max_iter=100, power=0.9):
    """Polynomial decay of learning rate
        :param init_lr is base learning rate
        :param iter is a current iteration
        :param lr_decay_iter how frequently decay occurs, default is 1
        :param max_iter is number of maximum iterations
        :param power is a polymomial power

    """
    #if iteraion % lr_decay_iter or iteraion > max_iter:
    #    return optimizer
    rate = int(epoch / 100)
    lr = init_lr * np.power(0.9, rate)
    #lr = init_lr*(1 - iteraion/max_iter)**power
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

    return lr

In [0]:
## process the date
data_process(path=path) ## generate "train_tomato.h5"

In [0]:

##                 the main training part (splitting data to trainng and testing)
#####   5 fold validation
for lt in loss_type:
    for f in range(5):
        iteration_folder = "iteration_HSCNNR_{}_8_{}".format(lt, str(f))
        model_folder = "models_HSCNNR_{}_8_{}".format(lt, str(f))
        #loss_type = "mrae" #"mrae"
        iteration_path = os.path.join(os.getcwd(), iteration_folder)
        if not os.path.exists(iteration_path):
            os.makedirs(iteration_path)
        ## new plot function with customed saving path
        
        def main():
            cudnn.benchmark = True
            ## network architecture
            rgb_features = 3
            hyper_features = 204
            ## load dataset
            print("\nloading dataset ...\n")
            #
            trainDataset = HyperDataset(crop_size=30)  ## here not the training data but the whole data set for this work
            # set the ratio of training and validation set
            validation_split = (1/3)
            
            dataset_len = len(trainDataset) #trainDataset
            indices = list(range(dataset_len))
            
            # Randomly splitting indices:
            val_len = int(np.ceil(validation_split * dataset_len))
            validation_idx = np.random.choice(indices, size=val_len, replace=False)
            train_idx = list(set(indices) - set(validation_idx))
            
            ## Defining the samplers for each phase based on the random indices:
            train_sampler = SubsetRandomSampler(train_idx)
            validation_sampler = SubsetRandomSampler(validation_idx)
            
            # Data Loader (Input Pipeline)
            train_data_loader = DataLoader(dataset=trainDataset,
                                           sampler=train_sampler,
                                           num_workers=1,  
                                           batch_size=8,
                                           shuffle=False,
                                           pin_memory=True)
            
            val_loader = DataLoader(dataset=trainDataset,
                                    sampler=validation_sampler,
                                    num_workers=1, 
                                    batch_size=1,
                                    shuffle=False,
                                   pin_memory=True)
            
            # Model               
            model = resblock(conv_relu_res_relu_block, 6, 3,204)
            # check whether it is possible to do multi-GPU processing
            if torch.cuda.device_count() > 1:
                model = nn.DataParallel(model)
            if torch.cuda.is_available():
                model.cuda()
          
            # Parameters, Loss and Optimizer
            start_epoch = 0
            end_epoch = 3000
            init_lr = 5e-3
            iteration = 0
            
            ## set the original values for the evaluation
            record_ts_loss = np.inf
            record_ts_MRAE = np.inf
            record_ts_RMSE = np.inf
            record_ts_SAM = np.inf
            # decide which loss function to use
            #criterion = nn.MSELoss()
            criterion = mrae_loss
            optimizer=torch.optim.Adamax(model.parameters(), lr=init_lr, betas=(0.9, 0.999), eps=1e-08, weight_decay=1e-4)
            
            model_path = os.path.join(os.getcwd(), model_folder)
            if not os.path.exists(model_path):
                os.makedirs(model_path)
            loss_csv = open(os.path.join(model_path, loss_type + '_loss_{}.csv'.format(str(f))), 'w+')
            
            log_dir = os.path.join(model_path,loss_type + '_train_{}.log'.format(str(f)))
            logger = initialize_logger(log_dir)
            
            # Resume
            resume_file ="" # can be specified if there is already trained model 
            if resume_file:
                print("=> loading checkpoint '{}'".format(resume_file))
                checkpoint = torch.load(os.path.join(model_path, resume_file))
                start_epoch = checkpoint['epoch']
                iteration = checkpoint['iter']
                model.load_state_dict(checkpoint['state_dict'])
                optimizer.load_state_dict(checkpoint['optimizer'])
               
            for epoch in range(start_epoch + 1, end_epoch): ## specify the number of epochs to run
                
                start_time = time.time()         
                tr_loss, tr_MRAE, tr_RMSE, tr_SAM, iteration, lr = train(train_data_loader, model, criterion, optimizer, iteration, init_lr, epoch)
                # here to implement constant learning rate
                end_time = time.time() # only record the training time
                epoch_time = end_time - start_time
                print(datetime.datetime.now())
                #lr=init_lr
                ts_loss, ts_MRAE, ts_RMSE,ts_SAM = validate(val_loader, model, criterion, epoch)
                
                # Save model
                # (1) ts_loss evaluation
                print("old_ts_MRAE", record_ts_MRAE, "old_ts_RMSE",record_ts_RMSE, "old_ts_SAM",record_ts_SAM)
        
                if ts_MRAE < record_ts_MRAE:
                    record_ts_MRAE = ts_MRAE
                    save_checkpoint_sp(model_path, epoch, iteration, model, optimizer, "ts_MRAE")
                    print("ts_MRAE")
                    print("updated_ts_MRAE", record_ts_MRAE, "updated_ts_RMSE",record_ts_RMSE, "updated_ts_SAM",record_ts_SAM)
        
                if ts_RMSE < record_ts_RMSE:  
                    record_ts_RMSE = ts_RMSE
                    save_checkpoint_sp(model_path, epoch, iteration, model, optimizer, "ts_RMSE")
                    print("ts_RMSE")
                    print("updated_ts_MRAE", record_ts_MRAE, "updated_ts_RMSE",record_ts_RMSE, "updated_ts_SAM",record_ts_SAM)
        
                if ts_SAM < record_ts_SAM:
                    record_ts_SAM = ts_SAM
                    save_checkpoint_sp(model_path, epoch, iteration, model, optimizer, "ts_SAM")
                    print("ts_SAM")
                    #print("                                  updated_min_val_RMSE",record_test_loss)
                    print("updated_ts_MRAE", record_ts_MRAE, "updated_ts_RMSE",record_ts_RMSE, "updated_ts_SAM",record_ts_SAM)
                    
                print("")
                # print loss 
        
                print ("fold: {}, Epoch:{}, Iter:{}, Time:{}, learning rate:{}, Train MRAE:{}, Train RMSE:{}, Train SAM:{}".format(
                    f, epoch, iteration, epoch_time, lr, tr_MRAE,tr_RMSE, tr_SAM))
                print ("Test MRAE:{}, Test RMSE:{}, Test SAM:{}".format(ts_MRAE, ts_RMSE, ts_SAM))
                # save loss( the content of "record loss" function is here)
                loss_csv.write('{},{},{},{},{},{},{},{},{},{},{}\n'.format(f, epoch, iteration, epoch_time, lr, tr_MRAE, tr_RMSE,
                                                                                                 tr_SAM, ts_MRAE, ts_RMSE, ts_SAM))
                loss_csv.flush()    
                loss_csv.close
                #record_loss_sp(loss_csv,epoch, iteration, epoch_time, lr, tr_loss,tr_MRAE, tr_RMSE_1,
                #                                                    tr_rRMSE_1, tr_RMSE_2,tr_rRMSE_2, tr_SAM, ts_loss, ts_MRAE,
                #                                                    ts_RMSE_1,ts_rRMSE_1,ts_RMSE_2,ts_rRMSE_2,ts_SAM)     
                logger.info("fold{},Epoch{}, Iter{}, Time:{}, learning rate:{}, Train MRAE:{},Train RMSE:{}, Train SAM:{}, Test MRAE:{},Test RMSE:{}, Test SAM:{}".format(
                    f, epoch, iteration, epoch_time, lr, tr_MRAE,tr_RMSE, tr_SAM, ts_MRAE,ts_RMSE,ts_SAM))
        
    # Training 
    def train(train_data_loader, model, criterion, optimizer, iteration, init_lr ,epoch):
        
        losses = AverageMeter()
        
        average_MRAE = 0.
        average_RMSE = 0. 
        average_SAM = 0.
    
        num=len(train_data_loader)
        print('num.{}'.format(num))
        optimizer.zero_grad()
    
        for i, (labels, images) in enumerate(train_data_loader):
            H = labels.size(2)
            W = labels.size(3)
            #
            labels = labels.cuda(async=True)
            images = images.cuda(async=True)
            #
            images = Variable(images)
            labels = Variable(labels)    
            
            # Decaying Learning Rate
            lr = poly_lr_scheduler(optimizer, init_lr, epoch, max_iter=968000, power=1.5) 
            iteration = iteration + 1
            #       
            output = model.forward(images)
            loss = criterion(labels, output)
            loss.backward()
            ## accumulate the loss for more than one batchs if necessary
            if (i+1)%1==0: # sepcify how many batches to accumulate the gradients
                #print("accumulated", j+1)
                optimizer.step()
                optimizer.zero_grad()
            #####
            MRAE = batch_MRAE(labels, output)
            RMSE = batch_RMSE(labels, output)
            SAM = batch_SAM(labels, output)
            
            average_MRAE    += MRAE.item()
            average_RMSE    += RMSE.item()
            average_SAM    += SAM.item()
        
        return losses.avg, average_MRAE/num, average_RMSE/num, average_SAM/num, iteration, lr
    
    # Validation
    
    def validate(val_loader, model, criterion, epoch):
            
        model.eval()
        losses = AverageMeter()
        xxx=0
        num = len(val_loader)
    
        average_MRAE    =0.
        average_RMSE    =0.
        average_SAM    =0.
    
        #v_accu= 0.
        for i, (target, input) in enumerate(val_loader):
            #
            H = target.size(2)
            W = target.size(3)
            #
            input = input.cuda(async=True)
            target = target.cuda(async=True)
            input_var = torch.autograd.Variable(input)
            target_var = torch.autograd.Variable(target)
    
            # compute output
            with torch.no_grad():
                output = model.forward(input_var)
            loss = criterion(output, target_var)
            #####
            MRAE = batch_MRAE(target_var, output)
            RMSE = batch_RMSE(target_var, output)
            SAM = batch_SAM(target_var, output)
            
            average_MRAE    += MRAE.item()
            average_RMSE    += RMSE.item()
            average_SAM    += SAM.item()
            #####
            ## generate a figure compare the reconstructed spectra and ground truth, every epoch
            if epoch%1==0:
                #print(i)
                if (i+2)%3==0:
                    xxx += 1
                    real_spectrum = target_var.data.cpu().numpy()[0,:,int(H/2),int(W/2)]
                    fake_spectrum = output.data.cpu().numpy()[0,:,int(H/2),int(W/2)]
                    I_spectrum = plot_spectrum(real_spectrum, fake_spectrum, "val_"+str(epoch),i)
                    print("val_num: " + str(xxx))
            #  record loss
            losses.update(loss.item())
        return losses.avg, average_MRAE/num, average_RMSE/num, average_SAM/num
    
    if __name__ == '__main__':
        main()


In [0]:
#

In [0]:
## here is to close the already open csv file in case the running process was stopped in the middle
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import os
loss_csv.close


In [0]:
## reconstruct hyperspectral images from smartphone RGB images in a loop

def get_reconstruction(input, num_split, dimension, model):
    """As the limited GPU memory split the input."""
    input_split = torch.split(input,  int(input.shape[3]/num_split), dim=dimension)
    output_split = []
    for i in range(num_split):
        var_input = Variable(input_split[i].cuda())
        var_output = model(var_input)
        output_split.append(var_output.data)
        if i == 0:
            output = output_split[i]
        else:
            output = torch.cat((output, output_split[i]), dim=dimension)
    return output

def reconstruction(rgb,model):
    """Output the final reconstructed hyperspectral images."""
    img_res = get_reconstruction(torch.from_numpy(rgb).float(),1, 3, model)
    img_res = img_res.cpu().numpy()*1
    img_res = np.transpose(np.squeeze(img_res))
    img_res_limits = np.minimum(img_res,1)
    img_res_limits = np.maximum(img_res_limits,0)
    return img_res_limits

######  
loss_type = ["mrae", "mse"]

for lt in loss_type:
    for i in range(5):
        iter_folder_rc = "SMt_iteration_HSCNNR_{}_8_{}".format(lt, str(i))  ## specify which folder to use
        loss_type = "{}_".format(lt) # specify the loss type 
        iter_path = os.path.join(os.getcwd(),iter_folder_rc)
        if not os.path.exists(iter_path):
            os.makedirs(iter_path)
        ##
        model_folder = "models_HSCNNR_{}_8_{}".format(lt,str(i))
        input_path = os.path.join(os.getcwd(), model_folder)
        if not os.path.exists(input_path):
            os.makedirs(input_path)
        ##
        out_folder = "SMt_out_HSCNNR_{}_8_{}".format(lt,str(i))
        out_path= os.path.join(os.getcwd(), out_folder)
        if not os.path.exists(out_path):
            os.makedirs(out_path)
        
        ## the model names
        loss_csv = open(os.path.join(out_path, loss_type + 'evaluation.csv'), 'w+')
        
        models_ids = [f for f in os.listdir(input_path) if f.endswith('.pkl')]
        
        for i in range(len(models_ids)):
            models_ids_i = models_ids[i] # the ith model name
            models_path = input_path
            models_path_i = os.path.join(models_path, models_ids_i)
            ## subtract the folder name
            eval_rc_i = "_".join(models_ids_i.split(".")[0].split("_")[4:])
            models_path_out_i = os.path.join(out_path, loss_type + eval_rc_i)
            
            if not os.path.exists(models_path_out_i):
                os.makedirs(models_path_out_i) 
            #read the model
            var_name = 'img' # the variable name used in mat file
            # load the weights
            save_point = torch.load(models_path_i)
            model_param = save_point['state_dict']
            model = resblock(conv_relu_res_relu_block,6,3,204)
            model.load_state_dict(model_param)
            model = model.cuda()
            model.eval()
            ## Reconstruct HSI of all test images
        
            #  load the images
            img_path = "input image folder path"
        
            png_files= [f for f in os.listdir(img_path) if f.endswith('.png')]
            
            device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
            
            for img_name in sorted(png_files):
                print (("img_name",img_name))
                hyper_name = img_name.replace("png", "h5")
                
                img_path_name = os.path.join(img_path, img_name)
                rgb = cv2.imread(img_path_name)
                rgb = cv2.rotate(rgb, cv2.ROTATE_90_CLOCKWISE)
                rgb=cv2.cvtColor(rgb, cv2.COLOR_BGR2RGB)
                
                H = rgb.shape[0]
                W = rgb.shape[1]
        
                rgb = normalize(np.float32(rgb))
                print(("rgb_or", rgb.shape))
                rgb = np.expand_dims(np.transpose(rgb,[2,0,1]), axis=0).copy()
                print(("rgb_ex", rgb.shape)) 
                ##
                
                #img_res1 = reconstruction(rgb,model)
                img_res = get_reconstruction(torch.from_numpy(rgb).float(),1, 3, model)
        
                # prepare to save the reconstructed HSI
                print(("img_res", img_res.shape))
                img_res_n = img_res.cpu().numpy()*1
                img_res_n = np.transpose(np.squeeze(img_res_n))
                # clip the data to 0 and 1 range
                img_res_n_limits = np.minimum(img_res_n,1)
                img_res_n_limits = np.maximum(img_res_n_limits,0)
        
                print(("img_res_n", img_res_n_limits.shape))
                # create the file name for .mat file
                mat_name = img_name.split(".")[0] + '.mat'
                print(("mat_name",mat_name))
        
                mat_dir= os.path.join(models_path_out_i, mat_name)
                print(mat_dir)
                save_matv73(mat_dir, var_name, img_res_n_limits)
         

In [0]:

##       using the trained model to reconstruct "redness image" from SMARTPHONE images

def get_reconstruction(input, num_split, dimension, model):
    """As the limited GPU memory split the input."""
    input_split = torch.split(input,  int(input.shape[3]/num_split), dim=dimension)
    output_split = []
    for i in range(num_split):
        var_input = Variable(input_split[i].cuda())
        var_output = model(var_input)
        output_split.append(var_output.data)
        if i == 0:
            output = output_split[i]
        else:
            output = torch.cat((output, output_split[i]), dim=dimension)
    
    return output

def reconstruction(rgb,model):
    """Output the final reconstructed hyperspectral images."""
    img_res = get_reconstruction(torch.from_numpy(rgb).float(),1, 3, model)
    img_res = img_res.cpu().numpy()*1
    img_res = np.transpose(np.squeeze(img_res))
    img_res_limits = np.minimum(img_res,1)
    img_res_limits = np.maximum(img_res_limits,0)
    return img_res_limits


## specify the model folder
#mse_models_path = glob.glob(os.path.join(os.getcwd(), mse_model_folder, '*.pkl'))
mse_models_path = "model path"

var_name = 'img' ## specify the variable name of the .mat file
# load the model weights
save_point = torch.load(mse_models_path)
model_param = save_point['state_dict']
model = resblock(conv_relu_res_relu_block,6,3,204)
model.load_state_dict(model_param)

model = model.cuda()
model.eval()

## Reconstruct HSI of all test images

img_path = 'input_path' # path of smartphone image
result_path = 'output_path' # path of reconstructed HSI of smartphone image
if not os.path.exists(result_path):
    os.makedirs(result_path)

png_files= [f for f in os.listdir(img_path) if f.endswith('.png')]

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


num = len(png_files)

for img_name in sorted(png_files):
    print (("img_name",img_name))
    hyper_name = img_name.replace("png", "h5")

    img_path_name = os.path.join(img_path, img_name)
    rgb = cv2.imread(img_path_name)
    rgb=cv2.cvtColor(rgb, cv2.COLOR_BGR2RGB)
    if rgb.shape[0]<rgb.shape[1]:
        rgb =  cv2.rotate(rgb, cv2.ROTATE_90_CLOCKWISE)

    rgb= cv2.resize(rgb, (512, 512)) # for change the resolution to fit the RAM of your computer

    H = rgb.shape[0] 
    W = rgb.shape[1] 

    rgb = normalize(np.float32(rgb))
    print(("rgb_or", rgb.shape))
    ## specify the placeholder for output image     
    rgb_i = np.expand_dims(np.transpose(rgb,[2,0,1]), axis=0).copy()
    print(("rgb_ex", rgb_i.shape)) 
    #img_res1 = reconstruction(rgb,model)
    img_res_i = get_reconstruction(torch.from_numpy(rgb_i).float(),1, 3, model)

    # prepare to save the reconstructed HSI
    print(("img_res", img_res_i.shape))
    img_res_i_n = img_res_i.cpu().numpy()*1
    print(("img_res_i_n", img_res_i_n.shape ))
    img_res_i_n = np.squeeze(img_res_i_n)
    # calculate redness of each pixel
    img_res_i_n_800 = img_res_i_n[137,:,:] # 800 nm 
    img_res_i_n_680 = img_res_i_n[97,:,:]  # 680 nm
    img_res_i_n_x = 1-np.abs((img_res_i_n_800 - img_res_i_n_680)/(img_res_i_n_800 + img_res_i_n_680))

    img_res_i_n_x = np.transpose(img_res_i_n_x,[1,0])
    print(("img_res_i_n_reshape", img_res_i_n_x.shape ))
    img_res_i_n_x_limits = np.minimum(img_res_i_n_x,1)
    img_res_i_n_x_limits = np.maximum(img_res_i_n_x,0)
    print(("img_res_i_n", img_res_i_n_x_limits.shape))
    
    mat_name = img_name.split(".")[0] + '.mat'
    print(("mat_name",mat_name))
    mat_dir= os.path.join(result_path, mat_name)
    print(mat_dir)
    save_matv73(mat_dir, var_name, img_res_i_n_x_limits)
    #print("file_name", file_name)  